In [1]:
import sys
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from datetime import datetime
from itertools import combinations
PREDICCION_REAL = True
MAXIMIZAR_HIPERPARAMETROS = False
PARAMETROS = {"booster":"gbtree", "max_depth":3, "eta": 0.1, "objective": "binary:logistic", "nthread":2,"gamma" : 50,"subsample" : 1}
RONDAS = 500
EARLY = 1

In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_csv("entrenamiento-listo.csv")
test = pd.read_csv("test-listo.csv")

In [3]:
#FILTRADO DE COLUMNAS - NO REMOVER STAGE O FECHA
entrenamiento = entrenamiento.drop(columns=['ID','Opportunity_Name','Sales_Contract_No'])
test = test.drop(columns=['ID','Opportunity_Name','Sales_Contract_No'])

In [4]:
if(PREDICCION_REAL):
    objetivo = (entrenamiento['Stage'] == 'Closed Won').astype(int)
    entrenamiento = entrenamiento.drop(columns=['Stage','Fecha'])
    
    d_entrenamiento = xgb.DMatrix(entrenamiento.values, objetivo.values)
    d_prueba = xgb.DMatrix(test.values)
    
    bst = xgb.train(PARAMETROS, d_entrenamiento, RONDAS)
    preds = bst.predict(d_prueba)

    resultados = test[['Opportunity_ID']].copy()
    resultados['Target'] = pd.Series(preds)
    #resultados = resultados.groupby('Opportunity_ID').mean()
    #resultados = resultados.reset_index()
    #resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))
    
    resultados.to_csv("prediccion.csv", index=False)
    print(resultados['Target'].count())

[18:13:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
2551


In [5]:
if(PREDICCION_REAL): sys.exit()

SystemExit: 

C:\Users\gianb\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
entrenamiento_label = (entrenamiento['Stage'] == 'Closed Won').astype(int)
test_label          = (test['Stage'] == 'Closed Won').astype(int)

set_entrenamiento = xgb.DMatrix(entrenamiento.drop(columns=['Stage','Fecha']),label = entrenamiento_label)
set_test  = xgb.DMatrix(test .drop(columns=['Stage','Fecha']),label = test_label)
evaluacion = [(set_test, 'eval'), (set_entrenamiento, 'train')]

modelo = xgb.train(PARAMETROS, set_entrenamiento, RONDAS, evaluacion, early_stopping_rounds=EARLY)

In [ ]:
prediccion = modelo.predict(set_test)

resultados = test[['Opportunity_ID']].copy()
resultados['Target'] = pd.Series(prediccion)
#resultados = resultados.groupby('Opportunity_ID').mean()
#resultados = resultados.reset_index()
#resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))

resultados.to_csv("prediccion.csv", index=False)
print(resultados['Target'].count())

prediccion = [1 if i > .5 else 0 for i in prediccion]

def metricas(objetivo, prediccion):
    matriz_conf = confusion_matrix(objetivo, prediccion)
    score = accuracy_score(objetivo, prediccion)
    reporte = classification_report(objetivo, prediccion)
    metricas = [matriz_conf, score, reporte]
    return(metricas)

metrics = metricas(test_label, prediccion)
print("Prediccion Test")
[print(i) for i in metrics]
print(skl.metrics.log_loss(test_label,prediccion))

prediccion = modelo.predict(set_entrenamiento)
prediccion = [1 if i > .5 else 0 for i in prediccion]
metricas = metricas(entrenamiento_label, prediccion)
print("Prediccion Train")
[print(i) for i in metricas]

In [ ]:
if(not MAXIMIZAR_HIPERPARAMETROS): sys.exit()

In [ ]:
min_log_loss = 1000
d = 1000
e = 1000
r = 1000
rondas = 500
ear = 1000
g = 1000
for depth in range(2,5):
    for eta in [0.1,0.3,0.5,0.7,0.9]:
        for early in range(1,5):
            for gamma in range(0,300,100):
                parametros = {"booster":"gbtree", "max_depth":depth, "eta": eta, "objective": "binary:logistic", "nthread":2,"gamma":gamma}
                modelo = xgb.train(parametros, set_entrenamiento, rondas, evaluacion,early_stopping_rounds = early)
                prediccion = modelo.predict(set_test)
                prediccion = [1 if i > .5 else 0 for i in prediccion]
                log_loss = skl.metrics.log_loss(test_label,prediccion)
                if (log_loss < min_log_loss):
                    min_log_loss = log_loss
                    d = depth
                    e = eta
                    r = rondas
                    g = gamma
                    ear = early


print("log: ",min_log_loss)                
print("depth: ",d)      
print("rondas",r)
print("gamma: ", g)
print("early: ",ear)

In [ ]:
print("log: ",min_log_loss)                
print("depth: ",d)      
print("eta: ",e)    
print("rondas",r)
print("gamma: ", g)
print("early: ",ear)